In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/mnt/sda/lst_space/QwQ-32B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# prompt = "How many r's are in the word \"strawberry\""
# messages = [
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )

# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=32768
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
# print(response)


/home/ksas/anaconda3/envs/overthink/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 14/14 [04:58<00:00, 21.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
def llm_generate(prompt, QwQ_tokenizer, QwQ_model):

    print(
        f"prompt: {prompt}",
    )

    # Round 1
    messages = [{"role": "user", "content": prompt}]

    text = QwQ_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = QwQ_tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = QwQ_model.generate(
        **model_inputs,
        max_new_tokens=32768
    )

    response_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ][0]
    think_token = QwQ_tokenizer("</think>", add_special_tokens=False).input_ids[0]
    think_token_idx = torch.where(response_ids == think_token)[0]


    think_ids = response_ids[:think_token_idx + 1]
    response_ids = response_ids[think_token_idx + 1:]

    response = tokenizer.batch_decode([model_inputs.input_ids[0], think_ids, response_ids], skip_special_tokens=True)

    input_tokens = model_inputs.input_ids.size(1)
    output_tokens = response_ids.size(0)
    # completion_tokens = response_ids.size(0) + think_ids.size(0)
    reasoning_tokens = think_ids.size(0)

    print("input_tokens: ", input_tokens)
    print("output_tokens: ", output_tokens)
    print("reasoning_tokens: ", reasoning_tokens)

    return {
        "text": response[2],
        "input tokens": input_tokens,
        "output tokens": output_tokens,
        "reasoning tokens": reasoning_tokens,
        "entire respose": response[1] + response[2],
    }

In [5]:
llm_generate("who are you", tokenizer, model)

prompt: who are you
user
who are you
assistant
<think>

input_tokens:  13
output_tokens:  76
reasoning_tokens:  133


{'text': "\n\nI am Qwen, a large language model developed by Alibaba Cloud. I'm designed to assist with a wide range of tasks, including answering questions, providing information, creative writing, logical reasoning, programming, and more. I can help with topics in technology, science, culture, and many other areas. How can I assist you today? Feel free to ask anything!",
 'input tokens': 13,
 'output tokens': 76,
 'reasoning tokens': 133,
 'entire respose': 'Okay, the user is asking "who are you." I need to provide a clear and concise response. Let me start by stating my name, Qwen. Then mention that I\'m a large language model developed by Alibaba Cloud. It\'s important to highlight my capabilities, like answering questions, providing information, and assisting with tasks. I should also list some areas I can help with, such as technology, science, culture, and more. Make sure to keep it friendly and approachable. Let me check if there\'s anything else. Oh, maybe add a line about bei